In [1]:
# Import Regex
import re
import phonenumbers
from phonenumbers import NumberParseException

In [2]:
# Identify different formats of phone number (US)
def is_phone_number(phone):
    patterns = [
        r'^\d{3}-\d{3}-\d{4}$',            # 123-456-7890
        r'^\(\d{3}\)\s\d{3}-\d{4}$',       # (123) 456-7890
        r'^\d{3}\.\d{3}\.\d{4}$',          # 123.456.7890
        r'^\+?\d{1,2}-\d{3}-\d{3}-\d{4}$', # +1-123-456-7890 or 1-123-456-7890
        r'^\d{10}$',                        # 1234567890

        # To match stranger formats
        r'^\+?\d{1,3}[\s.-]\d{3}[\s.-]\d{3}[\s.-]\d{4}$',  # +553 575-381.4440, +21.13 926 7829
        r'^00\d{1,3}[\s.-]?\d{2,3}[\s.-]?\d{3}[\s.-]?\d{4}$',  # 002716236.1690, 0049 48 490.5598
    ]

    # Check if the phone number matches any of the patterns
    for pattern in patterns:
        if re.match(pattern, phone):
            return True
    return False


In [3]:
# Test 1
test_inputs = [
    '(123) 456-7890',
    '123-456-7890',
    '482.950.3182',
    '+1-123-456-7890',
    '3489820455',
    '+553 575-381.4440', 
    '002716236.1690', 
    '0049 48 490.5598', 
    '+21.13 926 7829', # invalid
    '+41 15-968.9292', # invalid
    '1234-567-890'     # invalid
]

for text in test_inputs:
    print(is_phone_number(text))

True
True
True
True
True
True
True
True
False
False
False
